**What is the average ratio of transfer bags vs. checked bags across flights?**

In [27]:
import duckdb
con = duckdb.connect('../sql_databases/skyhack.duckdb')
con.execute("SHOW TABLES").df()

IOException: IO Error: Cannot open file "c:\users\harsh sethi\onedrive\desktop\skyhack\eda\..\sql_databases\skyhack.duckdb": The process cannot access the file because it is being used by another process.

File is already open in 
C:\Anaconda\python.exe (PID 15604)

In [ ]:
sql_query = '''
WITH bags_norm AS (
  SELECT DISTINCT
    UPPER(TRIM(company_id))                       AS company_id,
    CAST(flight_number AS VARCHAR)                AS flight_number,
    CAST(scheduled_departure_date_local AS DATE)  AS dep_date,
    UPPER(TRIM(scheduled_departure_station_code)) AS dep_iata,
    UPPER(TRIM(scheduled_arrival_station_code))   AS arr_iata,
    UPPER(TRIM(bag_type))                         AS bag_type_norm,
    UPPER(TRIM(bag_tag_unique_number))            AS bag_tag,
    CAST(bag_tag_issue_date AS DATE)              AS bag_date,
  FROM bagsData_cleaned
  WHERE bag_tag_unique_number IS NOT NULL
),
-- one handling unit per unique bag→arrival route on the outbound flight
handling_units AS (
  SELECT DISTINCT
    company_id, flight_number, dep_date, dep_iata, arr_iata, bag_tag, bag_type_norm
  FROM bags_norm
),
bag_counts AS (
  SELECT
    company_id, flight_number, dep_date, dep_iata,
    SUM(CASE WHEN bag_type_norm LIKE 'ORIGIN%' THEN 1 ELSE 0 END)                 AS checked_units,
    SUM(CASE WHEN bag_type_norm LIKE '%TRANSFER%' THEN 1 ELSE 0 END)               AS transfer_units_total  -- includes HOT TRANSFER
  FROM handling_units
  GROUP BY 1,2,3,4
),
ratios AS (
  SELECT
    company_id, flight_number, dep_date, dep_iata,
    checked_units, transfer_units_total,
    CASE WHEN checked_units > 0 THEN CAST(transfer_units_total AS DOUBLE) / checked_units END AS transfer_ratio
  FROM bag_counts
)
SELECT
  AVG(transfer_ratio) AS avg_transfer_to_checked_ratio,
  COUNT(*)            AS flights_considered
FROM ratios
WHERE checked_units > 0;
'''

con.execute(sql_query).df()


,avg_transfer_to_checked_ratio,flights_considered
0,3.638248,8316


In [ ]:
con.close()